In [49]:
%pip install openai requests bs4 pandas

  Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.2.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 1.1 MB/s eta 0:00:0000:0100:01
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Note: you may need to restart the kernel to use updated packages.


In [102]:
import os, json, time, openai, requests, urllib, urllib.parse
from bs4 import BeautifulSoup


os.makedirs('data/', exist_ok=True)

# Read API key from key.txt
with open("key.txt", "r") as f: api_key = f.read().strip()


def extract_links(html_source, prompt):
    system_prompt = (
        "You are an AI assistant that extracts relevant links from HTML source code "
        "based on the provided user prompt. Return the links as a JSON array."
    )    
    user_prompt = f"{prompt}\n\nHTML:\n{html_source}"  # Truncate to fit input limits
    client = openai.Client(api_key=api_key)  # Create a new client object 

    print("prompt", user_prompt)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.2
    )
    return response.choices[0].message.content.strip()



def fetch_webpage(url, timeout=10):
    response = requests.get(url, timeout=timeout)
    response.raise_for_status()
    return response.text

    

def fetch_subforums(url, timeout=10):
    html = fetch_webpage(url,timeout=timeout)
    soup = BeautifulSoup(html, 'html.parser')
    link_tuples = [(a.text.strip(), a['href']) for a in soup.select('#sf a')]
    return [(text, link) for text, link in link_tuples if link.startswith("https://www.librogame.net/index.php/forum/forum?id=")]



def fetch_threads(url, timeout=10):
    html = fetch_webpage(url,timeout=timeout)
    soup = BeautifulSoup(html, 'html.parser')
    link_tuples = [(a.text.strip(), a['href']) for a in soup.select('#vf a')]
    return [(text, link) for text, link in link_tuples if link.startswith("https://www.librogame.net/index.php/forum/topic?id=") and not "&p=" in link]


def fetch_threads_multiple(url):
    page = 1
    threads = []
    while True:
        new_threads = fetch_threads(url + "&p=" + str(page))        
        if new_threads and new_threads[0][0] in {t[0] for t in threads}:
            break
        threads.extend(new_threads)
        page += 1
    return threads


def fetch_posts(url, timeout=50):
    html = fetch_webpage(url,timeout=timeout)
    soup = BeautifulSoup(html, 'html.parser')
    for sig in soup.find_all(class_='ag_signature'):
        sig.decompose()
    posts = []
    for c in soup.find_all(class_='ag_postrow'):
        id = "ERROR"
        author = "ERROR"
        
        try: id = c.get('id')[len('postrow_area_'):]
        except: pass

        try: author = c.find(class_='ag_username').get_text(strip=True)
        except: pass

        posts.append([id, author, c.find(class_='ag_postmsg').get_text(strip=True)])
    return posts


def fetch_posts_multiple(url):
    page = 1
    posts = []
    while True:
        new_posts = fetch_posts(url + "&p=" + str(page))
        if new_posts and (new_posts[0][0], new_posts[0][1]) in {(p[0], p[1]) for p in posts}:
            break
        posts.extend(new_posts)
        page += 1
    return posts




[['152176',
  'Adriano',
  "Ore 00:07: la Terna dichiara chiusa la ricezione dei Corti 2021!Da come era iniziata, poteva esserci il dubbio che il tema avrebbe scoraggiato molti di voi, e invece... siamo davvero soddisfatti!Innanzitutto, ci teniamo a svelarvi una grande novità, forse la più grande di questa edizione: tuttii Corti verranno pubblicati in un’unica soluzione!Ebbene si: sono anni che se ne parla e già erano state sperimentate pubblicazioni accavallate o multiple, ma mai in un unico blocco. Sarà una bella sperimentazione, che eliminerà l’effetto “non sono sicuro su che voto dare perché non conosco ancora i corti successivi”. Avere una visione d’insieme, secondo noi, vi permetterà di stilare una classifica più accurata, oltre che leggere i racconti nell’ordine che più preferite.Facciamo quindi il punto della situazione, svelando cosa accadrà nei prossimi giorni:-giovedì 4 febbraio:pubblicazione in massa di tutti i Corti pervenuti. Verrà aperto un thread per ciascun Corto, le d

In [ ]:
# Ottieni la lista di tutti i thread, dal 2010 ad oggi, del forum "Librogame" su librogame.net

main_url = "https://www.librogame.net/index.php/forum/forum?id=15"

subforums = fetch_subforums(main_url)
print(subforums)

forum_thread_list = []

for forum_name, forum_link in subforums:
    threads = fetch_threads_multiple(forum_link)
    for thread_name, thread_link in threads:
        forum_thread_list.append([forum_name, thread_name, thread_link])

for item in forum_thread_list:
    print(item)

os.makedirs('data/', exist_ok=True)
with open("data/lista-threads.json", "w", encoding="utf-8") as f:
    json.dump(forum_thread_list, f, ensure_ascii=False, indent=2)

[('2025', 'https://www.librogame.net/index.php/forum/forum?id=31'), ('2024', 'https://www.librogame.net/index.php/forum/forum?id=30'), ('2023', 'https://www.librogame.net/index.php/forum/forum?id=29'), ('2022', 'https://www.librogame.net/index.php/forum/forum?id=28'), ('2021', 'https://www.librogame.net/index.php/forum/forum?id=27'), ('2020', 'https://www.librogame.net/index.php/forum/forum?id=26'), ('2019', 'https://www.librogame.net/index.php/forum/forum?id=25'), ('2018', 'https://www.librogame.net/index.php/forum/forum?id=24'), ('2017', 'https://www.librogame.net/index.php/forum/forum?id=23'), ('2016', 'https://www.librogame.net/index.php/forum/forum?id=22'), ('2015', 'https://www.librogame.net/index.php/forum/forum?id=16'), ('2014', 'https://www.librogame.net/index.php/forum/forum?id=17'), ('2013', 'https://www.librogame.net/index.php/forum/forum?id=18'), ('2012', 'https://www.librogame.net/index.php/forum/forum?id=19'), ('2011', 'https://www.librogame.net/index.php/forum/forum?id=

In [103]:
def escape_filename(name: str) -> str:
    return name.replace("/", "-").replace("\\", "-").replace(":", "-").replace("*", "-").replace("!", "").replace("?", "").replace("\"", "-").replace("<", "-").replace(">", "-").replace("|", "-")

with open('data/lista-threads.json', 'r') as file:
    forum_thread_list = json.load(file)

while True: 
    try:
        for forum, title, url in forum_thread_list:
            escaped_forum = escape_filename(forum)
            escaped_title = escape_filename(title)
            directory = f"data/threads/{escaped_forum}"
            os.makedirs(directory, exist_ok=True)
            file_path = f"{directory}/{escaped_title}.json"
            if os.path.exists(file_path):
                continue
            print(url)
            posts = fetch_posts_multiple(url)
            with open(file_path, 'w') as file:
                json.dump(posts, file, indent=4)
        break
    except Exception as e:
        print(f"Error occurred: {e}")
        time.sleep(15)  # Wait for 15 seconds before retrying


https://www.librogame.net/index.php/forum/topic?id=5785
https://www.librogame.net/index.php/forum/topic?id=5684
https://www.librogame.net/index.php/forum/topic?id=5711
https://www.librogame.net/index.php/forum/topic?id=5679
https://www.librogame.net/index.php/forum/topic?id=5718
https://www.librogame.net/index.php/forum/topic?id=5678
https://www.librogame.net/index.php/forum/topic?id=5676
https://www.librogame.net/index.php/forum/topic?id=5722
https://www.librogame.net/index.php/forum/topic?id=5707
https://www.librogame.net/index.php/forum/topic?id=5685
https://www.librogame.net/index.php/forum/topic?id=5677
https://www.librogame.net/index.php/forum/topic?id=5716
https://www.librogame.net/index.php/forum/topic?id=5713
https://www.librogame.net/index.php/forum/topic?id=5712
https://www.librogame.net/index.php/forum/topic?id=5690
https://www.librogame.net/index.php/forum/topic?id=5708
https://www.librogame.net/index.php/forum/topic?id=5697
https://www.librogame.net/index.php/forum/topic?